In [1]:
import jpype
# JVM başlat
jpype.startJVM(jpype.getDefaultJVMPath(),
         "-Djava.class.path=/Users/uzaycetin/Documents/driver/zemberek-tum-2.0.jar", "-ea")

# Türkiye Türkçesine göre çözümlemek için gerekli sınıfı hazırla
Tr = jpype.JClass("net.zemberek.tr.yapi.TurkiyeTurkcesi")
# tr nesnesini oluştur
tr = Tr()
# Zemberek sınıfını yükle
Zemberek = jpype.JClass("net.zemberek.erisim.Zemberek")
# zemberek nesnesini oluştur
zemberek = Zemberek(tr)

In [2]:
ornSonuc = zemberek.kelimeCozumle("karadan")
print(ornSonuc[0].kok().icerik())

kara


In [3]:
def kokbul(word = 'karasal'):
    kok = word
    ornSonuc = zemberek.kelimeCozumle(kok)
    try:
        kok = ornSonuc[0].kok().icerik()
    except:
        pass
    return kok

In [4]:
kokbul(word = 'karasal')

'kara'

In [5]:
kokbul(word = 'karasalnfbfb')

'karasalnfbfb'

In [8]:
kokbul('gezinti gelsene')

'gezinti gelsene'

In [9]:
txt = 'gezinti gelsene'
txt.split()

['gezinti', 'gelsene']

In [10]:
[kokbul(w) for w in txt.split()]

['gezinti', 'gel']

In [11]:
# Importing the libraries
import numpy as np
import pandas as pd
import re
import pickle 
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
nltk.download('stopwords')

# stopwordsleri sil
from nltk.corpus import stopwords
stop = stopwords.words('Turkish')

[nltk_data] Error loading stopwords: <urlopen error EOF occurred in
[nltk_data]     violation of protocol (_ssl.c:777)>


In [12]:
# Turkish Stopwords
with open('17bintweet/turkce-stop-words.txt') as file:  
    stw = file.read() 
stw = stw.split()
stw = [s.lower() for s in stw] 
stop += stw

In [13]:
def preprocessing(text):
    text = text.lower()
    # get rid of non-alphanumerical characters
    text = re.sub(r'\W', ' ', text) 
    # get rid of spaces
    text = re.sub(r'\s+', ' ', text) 
    # Correct mistakes 
    # zemberek!!
    
    # and do the stemming
    kelimeler = [kokbul(w) for w in text.split()]
    return " ".join([word for word in kelimeler if word not in stop])


In [14]:
test_data = pd.read_excel('17bintweet/test_tweets.xlsx', header=None)
test_data.head(3)

,0,1
0,Turkcell'e kızgınım. Ve bu kızgınlık sanırım a...,olumsuz
1,turkcell kadar şerefsiz misiniz ya,olumsuz
2,Burdan Turkcell'e sesleniyorum o 3 tl haram olsun,olumsuz


In [15]:
train_data = pd.read_excel('17bintweet/train_tweets.xlsx', header=None)
train_data.head(3)

,0,1
0,Ulan Wifi'ye bağlıyım ben. Ona bağlıyken Turkc...,olumsuz
1,20 dk 1 GB internet 500 mb sadece kaşar turkce...,olumsuz
2,Ayrıca turkcell superonline reklamı kadar da k...,olumsuz


In [20]:
data = pd.concat([train_data, test_data])
data.head()

,0,1
0,Ulan Wifi'ye bağlıyım ben. Ona bağlıyken Turkc...,olumsuz
1,20 dk 1 GB internet 500 mb sadece kaşar turkce...,olumsuz
2,Ayrıca turkcell superonline reklamı kadar da k...,olumsuz
3,Turkcell çok pahalı ya,olumsuz
4,Turkcell Kaş'ta internetin cekmiyor,olumsuz


In [21]:
text = train_data.iloc[0,0]
text = preprocessing(text)
text

'ulan wifi bağ bağ turkcell internet paket bit mesaj at öde'

In [22]:
# Creating the corpus
corpus = []
for i in range(len(data)):
    corpus.append(preprocessing(data.iloc[i,0]))  

In [23]:
data.iloc[:,1].value_counts()

olumsuz    6888
notr       5822
olumlu     4579
Name: 1, dtype: int64

In [24]:
# Creating the Tf-Idf model directly
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features= 10000)
vectorizer.fit(corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [25]:
X_train =  vectorizer.transform(corpus[:len(train_data)]).toarray()
X_test =  vectorizer.transform(corpus[len(train_data):]).toarray()

In [26]:
sentiment = {'olumsuz':0, 'notr':1, 'olumlu':2}
inv_sentiment = {v:k for k, v in sentiment.items()}
inv_sentiment

{0: 'olumsuz', 1: 'notr', 2: 'olumlu'}

In [27]:
y_train = np.array([sentiment[s] for s in train_data.iloc[:,1]])
y_test = np.array([sentiment[s] for s in test_data.iloc[:,1]])
y_train

array([0, 0, 0, ..., 0, 1, 2])

In [28]:
# Training the classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [29]:
# Testing model performance
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cm

array([[1075,  200,  102],
       [ 308,  708,  148],
       [ 206,  181,  529]])

In [24]:
accuracy_score(y_test, y_pred)

0.6687879664448945

In [25]:
raw_sample = ["Kampanya berbat.  kötü ya!", "güzel olmuş", "kötü değil", "müthiş harika sevdim"]
# Creating the corpus
test_sample = []
for i in range(len(raw_sample)):
    test_sample.append(preprocessing(raw_sample[i]))  

sample = vectorizer.transform(test_sample).toarray()
result = classifier.predict(sample)
[inv_sentiment[r] for r in result]

['olumsuz', 'olumlu', 'olumsuz', 'olumlu']

In [26]:
len(train_data), len(test_data)

(13832, 3457)

In [27]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
X_train.shape[1]

10000

# Keras

In [30]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

train_labels = keras.utils.to_categorical(y_train, 3)
test_labels = keras.utils.to_categorical(y_test, 3)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [37]:


model = Sequential()
model.add(Dense(1000, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 1000)              10001000  
_________________________________________________________________
dropout_6 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_7 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 3003      
Total para

In [38]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [40]:
history = model.fit(X_train, train_labels,
                    batch_size=10,
                    epochs=10,
                    verbose=2,
                    validation_data=(X_test, test_labels))

Train on 13832 samples, validate on 3457 samples
Epoch 1/10
 - 149s - loss: 0.0667 - acc: 0.9871 - val_loss: 2.8556 - val_acc: 0.6561
Epoch 2/10
 - 148s - loss: 0.0825 - acc: 0.9879 - val_loss: 2.5094 - val_acc: 0.6731
Epoch 3/10
 - 140s - loss: 0.0776 - acc: 0.9886 - val_loss: 2.6305 - val_acc: 0.6601
Epoch 4/10
 - 140s - loss: 0.0851 - acc: 0.9876 - val_loss: 3.2268 - val_acc: 0.6462
Epoch 5/10
 - 140s - loss: 0.1028 - acc: 0.9864 - val_loss: 2.6441 - val_acc: 0.6563
Epoch 6/10
 - 140s - loss: 0.0949 - acc: 0.9884 - val_loss: 2.8118 - val_acc: 0.6624
Epoch 7/10
 - 140s - loss: 0.0936 - acc: 0.9880 - val_loss: 3.2499 - val_acc: 0.6520
Epoch 8/10
 - 140s - loss: 0.0914 - acc: 0.9900 - val_loss: 3.1125 - val_acc: 0.6624
Epoch 9/10
 - 140s - loss: 0.0975 - acc: 0.9900 - val_loss: 3.5291 - val_acc: 0.6618
Epoch 10/10
 - 140s - loss: 0.0973 - acc: 0.9900 - val_loss: 3.4373 - val_acc: 0.6572
